# WorldView-2 Imagery Processing and STAC Catalog Creation

## Overview
This script processes **WorldView-2 imagery** (NTF format) and associated metadata (XML format) to produce:
1. Orthorectified imagery using RPC and DEM data.
2. Single-band Cloud Optimized GeoTIFFs (COGs) for each spectral band.
3. A local **SpatioTemporal Asset Catalog (STAC)** to organize and index the processed imagery.

## Key Steps
1. **Input Directories Setup**:
   - `input_dir`: Path to raw NTF and XML files.
   - `output_dir`: Directory for orthorectified imagery and COG outputs.
   - `stac_output`: Directory for the STAC catalog.

2. **Orthorectification**:
   - Utilizes GDAL's `gdalwarp` with RPCs to rectify NTF files.
   - Downloads DEM data using the `elevation` package to improve accuracy.

3. **Metadata Extraction**:
   - Parses XML files to extract acquisition time and bounding box coordinates.

4. **Single-Band COG Creation**:
   - Converts multiband orthorectified imagery into individual spectral bands as COGs using GDAL's `gdal_translate`.

5. **STAC Catalog Generation**:
   - Creates a local STAC catalog with:
     - Spatial and temporal metadata.
     - Assets for each spectral band, stored as COGs.

6. **Output**:
   - A fully self-contained STAC catalog stored in the specified `stac_output` directory.


## WorldView-2 Bands
The script supports the following WorldView-2 spectral bands:
- **Coastal** (Band 1)
- **Blue** (Band 2)
- **Green** (Band 3)
- **Yellow** (Band 4)
- **Red** (Band 5)
- **Red Edge** (Band 6)
- **NIR-1** (Band 7)
- **NIR-2** (Band 8)

## Outputs
1. **Orthorectified TIFFs** with DEM correction.
2. **COGs** for individual spectral bands.
3. A **STAC Catalog** organized with acquisition time, bounding box, and spectral band assets.


In [7]:
import os
os.environ["PATH"] = "/home/jovyan/geoai_veg_map/myenv/bin:" + os.environ["PATH"]
os.environ["PROJ_LIB"] = "/home/jovyan/geoai_veg_map/myenv/share/proj"


In [3]:
!which gdalbuildvrt

/home/jovyan/geoai_veg_map/myenv/bin/gdalbuildvrt


In [15]:
import os
import subprocess
import xml.etree.ElementTree as ET
from datetime import datetime
from pathlib import Path
from pystac import Catalog, Collection, Item, Asset, MediaType, CatalogType, Extent, SpatialExtent, TemporalExtent
import elevation

# Input directories
input_dir = os.path.abspath("imagery/worldview2/orig")  # Directory containing NTF and XML files
output_dir = os.path.abspath("imagery/worldview2/cog")  # Directory to store COG files
stac_output = "local_stac"             # Directory for the STAC catalog

os.makedirs(output_dir, exist_ok=True)
os.makedirs(stac_output, exist_ok=True)

# WorldView-2 Band Names
WV2_BAND_NAMES = [
    "coastal",  # Band 1
    "blue",     # Band 2
    "green",    # Band 3
    "yellow",   # Band 4
    "red",      # Band 5
    "rededge",  # Band 6
    "nir1",     # Band 7
    "nir2"      # Band 8
]

def orthorectify_ntf_with_rpc(ntf_path, output_path, dem_path=None, target_srs="EPSG:32610", res=1.84):
    cmd = [
        "gdalwarp",
        "--config", "NITF_J2K_DRIVER", "JP2OPENJPEG",
        "-of", "GTiff",
        "-rpc",
        "-t_srs", target_srs,
        "-tr", str(res), str(res),
        "-overwrite"
    ]
    if dem_path is not None:
        cmd += ["-to", f"RPC_DEM={dem_path}"]
    cmd += [ntf_path, output_path]

    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd)
    print(f"Orthorectification complete: {output_path}")

def extract_metadata_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # Extract datetime from <FIRSTLINETIME>
    first_line_time = root.find(".//FIRSTLINETIME").text
    acquisition_datetime = datetime.fromisoformat(first_line_time.replace("Z", "+00:00"))

    # Extract coords from BAND_C
    band_c = root.find(".//BAND_C")
    ULLON = float(band_c.find("ULLON").text)
    ULLAT = float(band_c.find("ULLAT").text)
    URLON = float(band_c.find("URLON").text)
    URLAT = float(band_c.find("URLAT").text)
    LRLON = float(band_c.find("LRLON").text)
    LRLAT = float(band_c.find("LRLAT").text)
    LLLON = float(band_c.find("LLLON").text)
    LLLAT = float(band_c.find("LLLAT").text)

    # Compute bounding box:
    min_lon = min(ULLON, URLON, LRLON, LLLON)
    max_lon = max(ULLON, URLON, LRLON, LLLON)
    min_lat = min(ULLAT, URLAT, LRLAT, LLLAT)
    max_lat = max(ULLAT, URLAT, LRLAT, LLLAT)
    bbox = [min_lon, min_lat, max_lon, max_lat]

    geometry = {
        "type": "Polygon",
        "coordinates": [[
            [ULLON, ULLAT],
            [URLON, URLAT],
            [LRLON, LRLAT],
            [LLLON, LLLAT],
            [ULLON, ULLAT]
        ]]
    }

    return acquisition_datetime, bbox, geometry

def download_dem_for_bbox(bbox, dem_path, buffer=0.05):
    # bbox = [west, south, east, north]
    west, south, east, north = bbox

    # Expand the bounding box by the specified buffer
    # This assumes the buffer is in the same units as the bbox coordinates (degrees).
    west -= buffer
    south -= buffer
    east += buffer
    north += buffer

    print(f"Downloading DEM for bbox with buffer: [{west}, {south}, {east}, {north}]")
    # Elevation product: 'SRTM1' (~30m), or 'SRTM3' (~90m).
    elevation.clip(bounds=(west, south, east, north), output=dem_path, product="SRTM3")


def create_single_band_cogs(multiband_path, output_dir):
    band_cogs = []
    for i, band_name in enumerate(WV2_BAND_NAMES, start=1):
        single_band_cog = os.path.join(output_dir, f"{Path(multiband_path).stem}_{band_name}.tif")
        subprocess.check_call([
            "gdal_translate",
            "-b", str(i),
            "-of", "COG",
            "-co", "COMPRESS=LZW",
            multiband_path,
            single_band_cog
        ])
        band_cogs.append((band_name, single_band_cog))
    return band_cogs

# Initialize STAC Catalog and Collection
catalog = Catalog(id="local-wv2-catalog", description="Local WV2 Imagery Catalog")
collection = Collection(
    id="worldview2-l1b",
    description="WorldView-2 Orthorectified Imagery",
    extent=None,  # We'll fill this after we add items
    license="proprietary"
)
catalog.add_child(collection)

# Find all NTF files and corresponding XML files
ntf_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".ntf")]

items = []
all_bboxes = []
all_times = []

for ntf_file in ntf_files:
    base_name = ntf_file.rsplit('.', 1)[0]
    xml_file = base_name + ".xml"
    ntf_path = os.path.join(input_dir, ntf_file)
    xml_path = os.path.join(input_dir, xml_file)

    if not os.path.exists(xml_path):
        print(f"No corresponding XML for {ntf_file}, skipping.")
        continue

    # Extract metadata first to get bbox
    acquisition_datetime, bbox, geometry = extract_metadata_from_xml(xml_path)
    print(bbox)
    # Download DEM for this bbox
    dem_path = os.path.join(output_dir, f"{base_name}_dem.tif")
    download_dem_for_bbox(bbox, dem_path)

    # Orthorectify
    ortho_tif = os.path.join(output_dir, f"{base_name}_ortho.tif")
    orthorectify_ntf_with_rpc(ntf_path, ortho_tif, dem_path=dem_path)

    # Create single-band COGs
    band_cogs = create_single_band_cogs(ortho_tif, output_dir)

    # Record spatial and temporal extents
    all_bboxes.append(bbox)
    all_times.append(acquisition_datetime)

    item_id = f"{base_name}_{acquisition_datetime.strftime('%Y%m%dT%H%M%S')}"
    item = Item(
        id=item_id,
        geometry=geometry,
        bbox=bbox,
        datetime=acquisition_datetime,
        properties={}
    )

    # Add each band as an asset
    for band_name, cog_path in band_cogs:
        item.add_asset(
            key=band_name,
            asset=Asset(
                href=os.path.relpath(cog_path),
                media_type=MediaType.COG,
                roles=["data"],
                title=f"{band_name.capitalize()} band"
            )
        )

    collection.add_item(item)
    items.append(item)



if items:
    minx = min(b[0] for b in all_bboxes)
    miny = min(b[1] for b in all_bboxes)
    maxx = max(b[2] for b in all_bboxes)
    maxy = max(b[3] for b in all_bboxes)
    temporal_start = min(all_times)
    temporal_end = max(all_times)

    collection.extent = Extent(
        SpatialExtent([[minx, miny, maxx, maxy]]),
        TemporalExtent([[temporal_start, temporal_end]])
    )
else:
    now = datetime.utcnow()
    collection.extent = Extent(
        SpatialExtent([[0, 0, 0, 0]]),
        TemporalExtent([[now, now]])
    )


# Save the catalog
catalog.normalize_hrefs(stac_output)
catalog.save(catalog_type=CatalogType.SELF_CONTAINED)
print(f"STAC Catalog saved to {stac_output}")


[-120.29258616, 34.76428285, -120.07671258, 34.93667514]
make: Entering directory '/home/jovyan/.cache/elevation/SRTM3'
make: Nothing to be done for 'download'.
make: Leaving directory '/home/jovyan/.cache/elevation/SRTM3'
make: Entering directory '/home/jovyan/.cache/elevation/SRTM3'
make: Nothing to be done for 'all'.
make: Leaving directory '/home/jovyan/.cache/elevation/SRTM3'
make: Entering directory '/home/jovyan/.cache/elevation/SRTM3'
cp SRTM3.vrt SRTM3.d89cf1f114ec44539e7e4c021a89f78e.vrt
make: Leaving directory '/home/jovyan/.cache/elevation/SRTM3'
make: Entering directory '/home/jovyan/.cache/elevation/SRTM3'
gdal_translate -q -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2 -projwin -120.34258616 34.986675139999996 -120.02671258000001 34.714282850000004 SRTM3.d89cf1f114ec44539e7e4c021a89f78e.vrt /home/jovyan/geoai_veg_map/imagery/worldview2/cog/WV02_20200524183756_10300100A89BD500_20MAY24183756-M1BS-504342154090_01_P002_dem.tif
rm -f SRTM3.d89cf1f114ec44539e7